In [1]:
import pandas as pd
import json

In [2]:
"hola"[0]

'h'

In [3]:
def add_type(codigo_comision):
    if(codigo_comision[0] != 'M' and codigo_comision[0] != 'T'):
        codigo_comision = 'M' + codigo_comision # Lamentablemente no tengo forma de saber si es M o T...
    return codigo_comision
def get_data():
    GSPREADHSEET_DOWNLOAD_URL = (
        "https://docs.google.com/spreadsheets/d/{gid}/export?format=csv&id={gid}".format
    )
    GID = '10ztDqFAi2HvbZNtOkGXK6dy-M22lXJkc6KHvpxVyOYQ'
    df = pd.read_csv(GSPREADHSEET_DOWNLOAD_URL(gid=GID))
    del df["Turno"]
    del df["Cupo"]
    df['Comisión'] = df["Comisión"].apply(add_type)
    return df.dropna()

In [4]:
def normalize_day(s):
    replacements = (
        ("á", "a"),
        ("é", "e"),
        ("í", "i"),
        ("ó", "o"),
        ("ú", "u"),
    )
    s = s.lower()
    for a, b in replacements:
        s = s.replace(a, b)
    return s
def nday(s):
    if s is None:
        return 0
    nday_dict = {'lunes':1, 'martes':2, 'miercoles':3, 'jueves':4, 'viernes':5, 'sabado':6}
    return nday_dict[normalize_day(s)]

In [5]:
df = get_data()

In [6]:
def get_horario():
    horarios = df["Dia"].str.split(' ', expand=True).drop([3,8], axis=1)
    horarios.set_axis(['tipo_1', 'dia_1', 'base_1', 'tope_1',
                       'tipo_2', 'dia_2', 'base_2', 'tope_2'], 
                    axis='columns', inplace=True)
    horarios['dia_1'] = horarios['dia_1'].apply(nday)
    horarios['dia_2'] = horarios['dia_2'].apply(nday)
    return horarios

In [7]:
horarios = get_horario()
del df['Dia']

In [8]:
df = df.join(horarios)

In [9]:
def format_clases(clases, row):
    clases.append({"dia": row["dia_1"],
                   "inicio": row["base_1"],
                   "fin": row["tope_1"]})
    if(row["dia_2"] != 0):
        clases.append({"dia": row["dia_2"],
               "inicio": row["base_2"],
               "fin": row["tope_2"]})
    return clases

def format_materia(cursos, row):
    cursos.append({ "codigo": row["Comisión"],
                    "materia": row["Comisión"].split("-")[0],
                    "docentes": row["Docente"]+' - '+row["Comisión"].split("-")[2],
                    "clases": format_clases([], row)})
    return cursos

In [10]:
DATA = {
    "cuatrimestre": "2C2022",
    "timestamp": 0,
}
DATA["materias"] = []
DATA["cursos"] = []

for index, row in df.iterrows():
    format_materia(DATA["cursos"], row)

for materia in df["Actividad / Materia"].unique():
    cursos = list(df[df["Actividad / Materia"] == materia]["Comisión"].values)
    cod = cursos[0].split("-")[0]
    DATA["materias"].append({"codigo": cod,
                             "cursos": cursos,
                             "nombre": materia})

In [11]:
dump = json.dumps(DATA, indent=2, ensure_ascii=False, sort_keys=True)
with open('src/data/horarios.js', 'w') as fw:
    fw.write("export const data = ")
    fw.write("\n") # Do not remove me. Make me easy to parse.
    fw.write(dump)    